# Breakthru
Player 1: 
- Frota: 8 navios amarelos e 1 navio-almirante localizados no centro do tabuleiro.
- Objetivo: Mover o navio-almirante para o perímetro do tabuleiro.

Player 2:
- Frota: 12 navios cinzas localizados nas bordas do tabuleiro.
- Objetivo: Capturar o navio-almirante. 

Gameplay:
- É escolhido aleatóriamente quem será o amarelo e quem será o cinza.
- O amarelo escolhe quem começa.
- Possibilidade de jogada 1: O jogador pode se mover pra qualquer espaço vazio verticalmente ou horizontalmente (Tal qual a torre do xadrez).
- Possibilidade de jogada 2: Mover um quadrado na diagonal para capturar uma peça do oponente (Tal qual um peao do xadrez).

# Tabuleiro
<div>
<img src="./tabuleiro.png" width="400"/>
</div>



In [1]:
import tkinter as tk

class BoardGameGUI:
    def __init__(self, master, lado, player, computer, begin):
        self.master = master
        self.player = player
        self.computer = computer
        self.lado = lado
        self.turn = begin
        self.create_board()

    def create_board(self):
        self.buttons = []
        self.board = [[None for _ in range(self.lado)] for _ in range(self.lado)]  # Representação do estado do tabuleiro
        for row in range(self.lado):
            button_row = []
            for col in range(self.lado):
                button = tk.Button(self.master, width=4, height=2, command=lambda r=row, c=col: self.on_button_click(r, c))
                button.grid(row=row, column=col)
                button_row.append(button)
            self.buttons.append(button_row)
        
        for i in range(2,5):
            self.place_piece(0, i, "Cinza")
            self.place_piece(i, 0, "Cinza")
            self.place_piece(i, 6, "Cinza")
            self.place_piece(6, i, "Cinza")

            self.place_piece(2, i, "Amarelo")
            self.place_piece(i, 2, "Amarelo")
            self.place_piece(i, 4, "Amarelo")
            self.place_piece(4, i, "Amarelo")

        self.place_piece(3, 3, "N")

    def on_button_click(self, row, col):
        if self.board[row][col] is not None:
            self.selected_piece = (row, col)
        elif self.selected_piece is not None:
            selected_row, selected_col = self.selected_piece
            self.move_piece(selected_row, selected_col, row, col)
            self.selected_piece = None     
        
    def place_piece(self, row, col, piece):
        self.board[row][col] = piece
        button = self.buttons[row][col]
        if piece == "Amarelo":
            button.config(bg="gold1")
        elif piece == "N":
            button.config(bg="goldenrod")
        elif piece == "Cinza":
            button.config(bg="gray60")
        else:
            button.config(bg="SystemButtonFace")
    
    def move_piece(self, row, col, new_row, new_col):
        piece = self.board[row][col]

        self.place_piece(row,col,None)              # Remove a peça da posição antiga
        self.place_piece(new_row,new_col,piece)     # Coloca a peça na nova posição

        if self.turn == "Amarelo":
            self.turn = "Cinza"
        else:
            self.turn = "Amarelo"

    def check_winner(self, board):
        # Checar se o Amarelo chegou no perimetro (Navio fugiu)
        for lado in range(self.lado):
            if board[0][lado] == "N":
                return "Amarelo"
            if board[6][lado] == "N":
                return "Amarelo"
            if board[lado][0] == "N":
                return "Amarelo"
            if board[lado][6] == "N":
                return "Amarelo"
            
        # Checar se nao tem N no tabuleiro (Navio capturado)
        for row in range(self.lado):
            for col in range(self.lado):
                if board[row][col] == "N":
                    return None
                
        return "Cinza"
    
    def disable_buttons(self):
        for row in self.buttons:
            for button in row:
                button.config(state="disabled")
    
    def enble_buttons(self):
        for row in self.buttons:
            for button in row:
                button.config(state="normal")
        

In [6]:
import random
import time

def main():
    window = tk.Tk()
    window.title("Breakthru")
    lado = 7
    text = ""

    if random.choice([1, 2]) == 1:
        player = "Cinza"
        computer = "Amarelo"
        text = "Computador é o Amarelo \nVocê é o Cinza"
    else:
        player = "Amarelo"
        computer = "Cinza"
        text = "Computador é o Cinza \nVocê é o Amarelo"

    info_label = tk.Label(window, text=text)
    info_label.grid(row=lado, columnspan=lado)
    
    begin = random.choice([player, computer])
    game = BoardGameGUI(window, lado, player, computer, begin)
    game.disable_buttons()
    window.update()
    time.sleep(2)

    while True:
        window.update()
        
        if game.turn == player:
            info_label.config(text="Sua vez")
            game.enble_buttons()
        else:
            info_label.config(text="Vez do computador")
            game.disable_buttons()
            window.update()
            time.sleep(2)
            game.turn = player

    window.destroy()

if __name__ == "__main__":
    main()

Quem começa é:  Amarelo
Amarelo
Amarelo
Amarelo


TclError: invalid command name ".!label"